In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn as sk
import csv

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Data = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')
PrecisionsMoyennes = pd.read_csv('../input/test-polynomial/precisions_moyennes.csv')

#train.drop(columns=['Name','Ticket','Cabin'])  ---> Pourquoi ça ne marche pas ?
#test.drop(columns=['Name','Ticket','Cabin'])

#SUPRESSION DES DONNEES INUTILES
Data.pop('Name')
Data.pop('Ticket')
Data.pop('Cabin')
test.pop('Name')
test.pop('Ticket')
test.pop('Cabin')

#Conversion des datas non-numériques

Data['Sex'] = Data['Sex'].apply({'male':-1, 'female':1}.get)
test['Sex'] = test['Sex'].apply({'male':-1, 'female':1}.get)
Data['Embarked'] = Data['Embarked'].apply({'C':0, 'S':1, 'Q':0.5}.get)
test['Embarked'] = test['Embarked'].apply({'C':0, 'S':1, 'Q':0.5}.get)

/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/test-polynomial/__results__.html
/kaggle/input/test-polynomial/precisions_moyennes.csv
/kaggle/input/test-polynomial/__notebook__.ipynb
/kaggle/input/test-polynomial/custom.css
/kaggle/input/test-polynomial/__output__.json


In [2]:
test

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,-1,34.5,0,0,7.8292,0.5
1,893,3,1,47.0,1,0,7.0000,1.0
2,894,2,-1,62.0,0,0,9.6875,0.5
3,895,3,-1,27.0,0,0,8.6625,1.0
4,896,3,1,22.0,1,1,12.2875,1.0
...,...,...,...,...,...,...,...,...
413,1305,3,-1,NaN,0,0,8.0500,1.0
414,1306,1,1,39.0,0,0,108.9000,0.0
415,1307,3,-1,38.5,0,0,7.2500,1.0
416,1308,3,-1,NaN,0,0,8.0500,1.0


In [3]:
#LISTE D'ALGORITHMES DE PREDICTION
from sklearn import linear_model
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB

Ridge = linear_model.RidgeClassifier()
LinReg = linear_model.LogisticRegression()
SDG = linear_model.SGDClassifier() #Stochastic Gradient Decent
PassiveAgressive = linear_model.PassiveAggressiveClassifier()
LDA = LinearDiscriminantAnalysis()
QDA = QuadraticDiscriminantAnalysis()
Tree = tree.DecisionTreeClassifier()

Mlpc_R_L = MLPClassifier(activation = 'relu', solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, 2), random_state=1) # Neural Network solver = lbfgs, activation = relu
Mlpc_R_S = MLPClassifier(activation = 'relu', solver='sgd', max_iter=1000, alpha=1e-5, hidden_layer_sizes=(10, 2), random_state=1) # Neural Network solver = sgd, activation = relu
Mlpc_R_A = MLPClassifier(activation = 'relu', solver='adam', max_iter=1000, alpha=1e-5, hidden_layer_sizes=(10, 2), random_state=1) # Neural Network solver = adam, activation = relu
Mlpc_T_L = MLPClassifier(activation = 'tanh', solver='lbfgs', max_iter=10000, alpha=1e-5, hidden_layer_sizes=(10, 2), random_state=1) # Neural Network solver = lbfgs, activation = tanh
Mlpc_T_S = MLPClassifier(activation = 'tanh', solver='sgd', max_iter=1000, alpha=1e-5, hidden_layer_sizes=(10, 2), random_state=1) # Neural Network solver = sgd, activation = tanh
Mlpc_T_A = MLPClassifier(activation = 'tanh', solver='adam', max_iter=1000, alpha=1e-5, hidden_layer_sizes=(10, 2), random_state=1) # Neural Network solver = adam, activation = tanh
Mlpc_L_L = MLPClassifier(activation = 'logistic', solver='lbfgs', max_iter=10000, alpha=1e-5, hidden_layer_sizes=(10, 2), random_state=1) # Neural Network solver = lbfgs, activation = logistic
Mlpc_L_S = MLPClassifier(activation = 'logistic', solver='sgd', max_iter=1000, alpha=1e-5, hidden_layer_sizes=(10, 2), random_state=1) # Neural Network solver = sgd, activation = logistic
Mlpc_L_A = MLPClassifier(activation = 'logistic', solver='adam', max_iter=1000, alpha=1e-5, hidden_layer_sizes=(10, 2), random_state=1) # Neural Network solver = adam, activation = logistic

SVC = sk.svm.SVC()
Kneighbors = sk.neighbors.KNeighborsClassifier()
GaussProcess = GaussianProcessClassifier()
GaussNB = GaussianNB()
RandomForest = RandomForestClassifier()
AdaBoost = AdaBoostClassifier()


nom_algo = ['Ridge', 'LinReg', 'SDG', 'PassiveAgressive', 'LDA', 'QDA', 'Tree', 'Mlpc_R_L', 'Mlpc_R_S', 'Mlpc_R_A', 'Mlpc_T_L', 'Mlpc_T_S', 'Mlpc_T_A', 'Mlpc_L_L', 'Mlpc_L_S', 'Mlpc_L_A', 'SVC', 'Kneighbors', 'GaussProcess', 'GaussNB', 'RandomForest', 'AdaBoost']
liste_algos=[Ridge, LinReg, SDG, PassiveAgressive, LDA, QDA, Tree, Mlpc_R_L, Mlpc_R_S, Mlpc_R_A, Mlpc_T_L,Mlpc_T_S, Mlpc_T_A, Mlpc_L_L, Mlpc_L_S, Mlpc_L_A, SVC,Kneighbors, GaussProcess, GaussNB, RandomForest, AdaBoost]
algorithmes = pd.DataFrame({'Nom':nom_algo, 'Algorithme':liste_algos})
algorithmes.set_index('Nom', inplace=True)

In [4]:
col_degres=[]
col_nom_algo=[]
precisions_moyennes=PrecisionsMoyennes.copy()
for nom in nom_algo:
    col_degres = col_degres+[1, 2, 3, 4, 5, 6]
    col_nom_algo=col_nom_algo+[nom]*6
precisions_moyennes.insert(0, 'Algorithme', col_nom_algo)
precisions_moyennes.insert(1, 'Degre', col_degres)


In [5]:
def data_interp(data):
# GENERATION ET NORMALISATION DES DONNEES
    dat = data.copy()
    y=pd.DataFrame()
    if 'Survived' in dat.columns:
        y = dat['Survived'].copy()
    Id = dat['PassengerId'].copy()
    

    if 'Survived' in dat.columns:
        X=dat.drop(columns = ['Survived','PassengerId'])
    else: X=dat.drop(columns = ['PassengerId'])

    X_col=list(X.columns)
# NORMALISATION
    X = (X - X.mean()) / (X.max() - X.min())
    
# MODIFICATION : NAN->INTERPOLATION
    if sum(X.isnull().values.ravel()) != 0:
        from sklearn.experimental import enable_iterative_imputer
        from sklearn.impute import IterativeImputer
        imp = IterativeImputer(random_state=0)
        X = imp.fit_transform(X)
    
    X=pd.DataFrame(X, columns=X_col)
    print(X)
    
    return [X, Id, y] if 'Survived' in dat.columns else [X, Id]

In [6]:
def data_mean(data):
# GENERATION ET NORMALISATION DES DONNEES
    dat = data.copy()
    if 'Survived' in dat.columns:
        y = dat['Survived'].copy()
    Id = dat['PassengerId'].copy()
    if 'Survived' in dat.columns:
        X=dat.drop(columns = ['Survived','PassengerId'])
    else: X=dat.drop(columns = ['PassengerId'])
    
# MODIFICATION : NAN->MEAN
    colonnes_a_corriger = [col for col in dat.columns if dat[col].isnull().any()]
    for col in colonnes_a_corriger:
        colonnes = X[col].mean()
        X[col]=X[col].fillna(colonnes)
    
# NORMALISATION
    X = (X - X.mean()) / (X.max() - X.min())
    
    return [X, Id, y] if 'Survived' in dat.columns else [X, Id]
        



In [7]:
def data_suppr(data):
# GENERATION ET NORMALISATION DES DONNEES
    dat = data.copy()
    dat.dropna(inplace=True)
    if 'Survived' in dat.columns:
        y = dat['Survived'].copy()
    Id = dat['PassengerId'].copy()
    if 'Survived' in dat.columns:
        X=dat.drop(columns = ['Survived','PassengerId'])
    else: X=dat.drop(columns = ['PassengerId'])
    
# NORMALISATION
    X = (X - X.mean()) / (X.max() - X.min())
    
    return [X, Id, y] if 'Survived' in dat.columns else [X, Id]
    



In [8]:
# CROSS-VALIDATION SUR LE DEGRE
def precision_poly(X_train_initial, y_train, X_cross_initial, y_cross, col_poly, algo, degre_max=10, nb_iter=10):
    # ENTREES : X_train_initial et y_train : données et résultats du train set | X_cross_initial et y_cross: données et résultats du set de cross-validation | 
    #           col_fixes : colonnes des données qu'on veut à des degrés supérieurs | degre_max : exposant maximum du polynome des données |
    #           nb_iter : nombre d'itérations pour le calcul moyen de la précision
    #
    #SORTIE : DataFrame contenant les colonnes : Degré (degré du polynome des données) (index)
    #                                            Precision_train : précision moyenne sur le set de training
    #                                            Precision_cross : précision moyenne sur le set de cross-vaisation
    
    train_prec = []
    cross_prec = []
    
    for d in range(1,degre_max+1):
        X_train = Polynomiation(X_train_initial, col_poly, d)
        X_cross = Polynomiation(X_cross_initial, col_poly, d)       
        [p_train, p_cross] = PrecisionMoyenne_train_cross(X_train, y_train, X_cross, y_cross, algo, nb_iter)
        
        train_prec.append(p_train)
        cross_prec.append(p_cross)

    prec_degre = pd.DataFrame({'Degre':list(range(1,degre_max+1)), 'Précision_Train':train_prec, 'Précision_Cross':cross_prec})
    prec_degre.set_index('Degre', inplace=True) 
    
    return prec_degre

In [9]:
def PrecisionMoyenne_train_cross(X_train_entree, y_train_entree, X_cross_entree, y_cross_entree, algo, nb_iter=10): 
#Renvoie la précision moyenne sur nb_iter itérations de l'algorithme sur les données de de train et de cross-validation 
#(si l'algorithme est déterministe, la précision est calculée seulement sur 1 iteration)  
    
    algorithme=algorithmes.loc[algo, 'Algorithme']
    
    X_train=X_train_entree.copy()
    y_train=y_train_entree.copy()
    X_cross=X_cross_entree.copy()
    y_cros=y_cross_entree.copy()

    prec_train = []
    prec_cross = []
    

    for it in range(nb_iter):
        algorithme.fit(X_train, y_train)
        prediction_train = np.array(algorithme.predict(X_train))
        precision_train = (prediction_train==y_train).mean()
        prec_train.append(precision_train)
        
        prediction_cross = np.array(algorithme.predict(X_cross))
        precision_cross = (prediction_cross==y_cross).mean()
        prec_cross.append(precision_cross)
        if prec_cross[it]==prec_cross[it-1]: break
    
    precision_moyenne_train = np.array(prec_train).mean()
    precision_moyenne_cross = np.array(prec_cross).mean()
    
    return [precision_moyenne_train, precision_moyenne_cross]

In [10]:
# PREPARATION DES DONNEES POUR CROSS-VALIDATION ET TEST
# Entrée : Données à séparer en groupe train / cross / test & Méthode de traitement des données manquante
#Sortie : Dataframe contenant les données, l'identifiant et le résultat attendu pour chaque groupe Train, Cross et TestCross

def preparation(data, methode):
    ratio_TrainReste = 0.8
    ratio_CrossTest = 1

    [train, reste]=Split(data, ratio_TrainReste)
    [cross, testcross]=Split(reste, ratio_CrossTest)
    Train = [X_train_total, Id_train, y_train] = methode(train)
    Cross = [X_cross_total, Id_cross, y_cross] = methode(cross)
    TestCross= [X_testcross, Id_testcross, y_testcross] = methode(testcross)
    
    dat = pd.DataFrame({'Donnee':['X', 'Id', 'y'], 'Train':Train, 'Cross':Cross, 'TestCross':TestCross})
    dat.set_index('Donnee', inplace=True)
    return dat

In [11]:
 def Polynomiation(donnee, col_polynomes, deg):
    col_fixes=set(donnee.columns.values.tolist())-set(col_polynomes)
    donneePoly = donnee[col_fixes].copy()
    donneePoly['Sex*Age']=donnee['Sex'].mul(donnee['Age'])
    donneePoly['Sex*Embarked']=donnee['Sex'].mul(donnee['Embarked'])
    donneePoly['Sex*Pclass']=donnee['Sex'].mul(donnee['Pclass'])

    for degre in range(1,deg+1):
        for col in col_polynomes :
            donneePoly[col+'^'+str(degre)]=donnee[col].pow(degre)

    return donneePoly

In [12]:


# ----- SEPARATION POUR CROSS-VALIDATION -----
def Split(donnee, ratio):

    #SHUFFLE
    donneesplit=donnee.sample(frac=1).copy()

    # SEPARATION
    part1 = donneesplit[:int(len(Data)*ratio)]
    part2 = donneesplit[int(len(Data)*ratio):]
    return [part1, part2]



In [13]:
''' # ------------ CROSS-VALIDATION : MOYENNE DES ALGORITHMES ------------ 

Methode = data_interp
noms_algos = algorithmes.index
degre_max=6
nb_iter=10

donnees_preparees = preparation(Data, Methode)
[X_train_initial, Id_train, y_train] = donnees_preparees['Train']
[X_cross_initial, Id_cross, y_cross] = donnees_preparees['Cross']
[X_testcross, Id_testcross, y_testcross] = donnees_preparees['TestCross']

col_poly=['Pclass', 'Age','SibSp', 'Parch', 'Fare']

results = {}

for d in range(1,degre_max+1):
    for i in range(len(nom_algo)):
        algo = noms_algos[i]
        prec = precision_poly(X_train_initial, y_train, X_cross_initial, y_cross, col_poly, algo, degre_max, nb_iter)
        results[algo]=prec
            
precisions = pd.concat(results)  '''

" # ------------ CROSS-VALIDATION : MOYENNE DES ALGORITHMES ------------ \n\nMethode = data_interp\nnoms_algos = algorithmes.index\ndegre_max=6\nnb_iter=10\n\ndonnees_preparees = preparation(Data, Methode)\n[X_train_initial, Id_train, y_train] = donnees_preparees['Train']\n[X_cross_initial, Id_cross, y_cross] = donnees_preparees['Cross']\n[X_testcross, Id_testcross, y_testcross] = donnees_preparees['TestCross']\n\ncol_poly=['Pclass', 'Age','SibSp', 'Parch', 'Fare']\n\nresults = {}\n\nfor d in range(1,degre_max+1):\n    for i in range(len(nom_algo)):\n        algo = noms_algos[i]\n        prec = precision_poly(X_train_initial, y_train, X_cross_initial, y_cross, col_poly, algo, degre_max, nb_iter)\n        results[algo]=prec\n            \nprecisions = pd.concat(results)  "

In [14]:
'''# PREDICTION MOYENNE 

Methode = data_interp
noms_algos = algorithmes.index
degre_max = 6

donnees_preparees = preparation(Data, Methode)
[X_train_initial, Id_train, y_train] = donnees_interp(['Train'])
[X_cross_initial, Id_cross, y_cross] = donnees_preparees['Cross']

col_poly=['Pclass', 'Age','SibSp', 'Parch', 'Fare']

predictions = []

for i in range(len(nom_algo)):
    for d in range(1,degre_max+1):
        algo = noms_algos[i]
        algorithme=algorithmes.loc[algo, 'Algorithme']
        
        X_train = Polynomiation(X_train_initial, col_poly, d)
        X_cross = Polynomiation(X_cross_initial, col_poly, d) 
        
        algorithme.fit(X_train, y_train)
        pred_degre_algo = np.array(algorithme.predict(X_cross))
        predictions.append(pred_degre_algo)
        

        
predictions=np.array(predictions)
predictions[predictions==0]=-1

prediction_moyenne = np.array(precisions_moyennes['Précision_Cross']).dot(predictions)
prediction_moyenne[prediction_moyenne>0]=1
prediction_moyenne[prediction_moyenne<0]=0


precision_finale = (prediction_moyenne==y_cross).mean()'''



"# PREDICTION MOYENNE \n\nMethode = data_interp\nnoms_algos = algorithmes.index\ndegre_max = 6\n\ndonnees_preparees = preparation(Data, Methode)\n[X_train_initial, Id_train, y_train] = donnees_interp(['Train'])\n[X_cross_initial, Id_cross, y_cross] = donnees_preparees['Cross']\n\ncol_poly=['Pclass', 'Age','SibSp', 'Parch', 'Fare']\n\npredictions = []\n\nfor i in range(len(nom_algo)):\n    for d in range(1,degre_max+1):\n        algo = noms_algos[i]\n        algorithme=algorithmes.loc[algo, 'Algorithme']\n        \n        X_train = Polynomiation(X_train_initial, col_poly, d)\n        X_cross = Polynomiation(X_cross_initial, col_poly, d) \n        \n        algorithme.fit(X_train, y_train)\n        pred_degre_algo = np.array(algorithme.predict(X_cross))\n        predictions.append(pred_degre_algo)\n        \n\n        \npredictions=np.array(predictions)\npredictions[predictions==0]=-1\n\nprediction_moyenne = np.array(precisions_moyennes['Précision_Cross']).dot(predictions)\nprediction_m

In [15]:
# PREDICTION MOYENNE : ECHANTILLON TEST - pour soumission 

Methode = data_interp
noms_algos = algorithmes.index
degre_max = 6

[X_train_initial, Id_train, y_train] = data_interp(Data)
[X_test_initial, Id_test]=data_interp(test)

col_poly=['Pclass', 'Age','SibSp', 'Parch', 'Fare']

predictions = []

for i in range(len(nom_algo)):
    for d in range(1,degre_max+1):
        algo = noms_algos[i]
        algorithme=algorithmes.loc[algo, 'Algorithme']
        
        X_train = Polynomiation(X_train_initial, col_poly, d)
        X_test = Polynomiation(X_test_initial, col_poly, d) 
        
        algorithme.fit(X_train, y_train)
        pred_degre_algo = np.array(algorithme.predict(X_test))
        predictions.append(pred_degre_algo)
        

        
predictions=np.array(predictions)
predictions[predictions==0]=-1

prediction_test = np.array(precisions_moyennes['Précision_Cross']).dot(predictions)

prediction_test[prediction_test<0]=int(0)
prediction_test[prediction_test>0]=int(1)

sortie = pd.DataFrame({'PassengerId' : Id_test, 'Survived' : prediction_test}).astype(int)
sortie.to_csv('prediction_moyenne.csv', index=False)

       Pclass       Sex       Age     SibSp     Parch      Fare  Embarked
0    0.345679 -0.352413 -0.096747  0.059624 -0.063599 -0.048707  0.232283
1   -0.654321  0.647587  0.104309  0.059624 -0.063599  0.076277 -0.767717
2    0.345679  0.647587 -0.046483 -0.065376 -0.063599 -0.047390  0.232283
3   -0.654321  0.647587  0.066611  0.059624 -0.063599  0.040786  0.232283
4    0.345679 -0.352413  0.066611 -0.065376 -0.063599 -0.047146  0.232283
..        ...       ...       ...       ...       ...       ...       ...
886 -0.154321 -0.352413 -0.033917 -0.065376 -0.063599 -0.037484  0.232283
887 -0.654321  0.647587 -0.134445 -0.065376 -0.063599 -0.004302  0.232283
888  0.345679  0.647587 -0.125699  0.059624  0.269734 -0.017087  0.232283
889 -0.654321 -0.352413 -0.046483 -0.065376 -0.063599 -0.004302 -0.767717
890  0.345679 -0.352413  0.028913 -0.065376 -0.063599 -0.047731 -0.267717

[891 rows x 7 columns]
       Pclass       Sex       Age     SibSp     Parch      Fare  Embarked
0    0.367225 

/opt/conda/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/opt/conda/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/opt/conda/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/opt/conda/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_resu

In [16]:
#with pd.option_context('display.max_rows', None, 'display.max_columns', None): print(pd.DataFrame(predictions))